# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [27]:
# import libraries
import pandas as pd
import numpy as np
import re
from sqlalchemy import create_engine
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import classification_report
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk import pos_tag
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split,GridSearchCV 
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from gensim.sklearn_api import W2VTransformer
import tensorflow as tf
import tensorflow.keras
from gensim.models import Word2Vec
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.optimizers import SGD
import pickle

In [28]:
# load data from database
engine = create_engine('sqlite:///figure_8_db.db')
df = pd.read_sql_table(con = engine , table_name = 'Message_label')
X = df.message 
Y = df.drop(['id', 'message','original','genre'], axis=1)
Y.shape

(26177, 36)

### 2. Write a tokenization function to process your text data

In [29]:
def tokenize(text):
    # First remove punctation and lowercase all letters
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    clean_tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    stemmer = PorterStemmer()
    
    # lemmatize, stem and remove stop words
    clean_tokens = [stemmer.stem(lemmatizer.lemmatize(word)) for word in clean_tokens]
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [30]:

class KN():
    def KerasNetwork():
        clf = Sequential()
        clf.add(Dense(64, activation='relu'))
        clf.add(Dense(64, activation='relu'))
        clf.add(Dense(36, activation='softmax'))
        clf.compile(loss='categorical_crossentropy', optimizer=SGD(), metrics=["accuracy"])
        return clf

    def fit(self, x, y):
        return self.fit(x,y)

    def transform(self):
        return self
    
    def predict(self, X):
        return self.predi
        

In [31]:
pipeline2  = Pipeline(
        [('vect',CountVectorizer(tokenizer=tokenize)),
         ('tfidf', TfidfTransformer()),
         ('clf' , MultiOutputClassifier(RandomForestClassifier())   )
        ]
    )

In [38]:
pipeline  = Pipeline(
        [('vect',CountVectorizer(tokenizer=tokenize)),
         ('tfidf',TfidfTransformer()),
         ('clf' , MultiOutputClassifier(MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(6, 3), random_state=1)) )  
        ]
    )
def build_model():
    pipeline  = Pipeline(
            [('vect',CountVectorizer(tokenizer=tokenize)),
             ('tfidf',TfidfTransformer()),
             ('clf' , MultiOutputClassifier(MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(6, 3), random_state=1)) )  
            ]
        )
    parameters ={
        'clf__estimator__hidden_layer_sizes':( (32,3), (32,4), (64,3)),
        #'vect__ngram_range': ((1, 1), (1, 2)),
        #'vect__max_df': (0.5,  1.0),
        'vect__max_features': (10000),
        'clf__estimator__alpha' : (0.00001 , 0.001 )

    }
    cv  = GridSearchCV(pipeline, param_grid=parameters,verbose=3,n_jobs=-1,cv=2)
    return cv
def display_results(y_test, y_pred):
    for i,j in enumerate(y_test.columns):
        print(j)
        confusion_mat = confusion_matrix(y_test[j], y_pred[:,i])
        accuracy = (y_pred[:,i] == y_test[j]).mean()
        print(classification_report(y_test[j], y_pred[:,i]))
        print("Labels:", j)
        print("Confusion Matrix:\n", confusion_mat)
        print("Accuracy:", accuracy)

In [33]:
#pipeline3  = Pipeline(
#        [('vect',CountVectorizer(tokenizer=tokenize)),
#         ('tfidf',TfidfTransformer()),
#         ('clf' , KerasClassifier( build_fn=KerasNetwork, nb_epoch=2) )  
#        ]
#    )

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [56]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)
#model_NN=pipeline3.fit(X_train,y_train)
model_perceptron=pipeline.fit(X_train,y_train)
y_pred_perceptron = model_perceptron.predict(X_test)
model_RF=pipeline2.fit(X_train,y_train)
y_pred_RF = model_RF.predict(X_test)
print( len(y_pred_perceptron) , len(y_pred_perceptron[0]))
print(y_pred_RF.mean(axis=1))
print(model_perceptron.predict(['A Comitee in Delmas 19, Rue ( street ) Janvier, Impasse Charite #2. We have about 500 people in a temporary shelter and we are in dire need of Water, Food, Medications, Tents and Clothes. Please stop by and see us.']))
#f1 = multioutput_fscore(y_test,y_pred, beta = 1)
overall_accuracy_perceptron = (y_pred_perceptron == y_test).mean().mean()
overall_accuracy_RF = (y_pred_RF == y_test).mean().mean()
print('Average overall accuracy perceptron {0:.2f}% \n'.format(overall_accuracy_perceptron*100))
print('Average overall accuracy random forest {0:.2f}% \n'.format(overall_accuracy_RF*100))
#display_results(y_test, y_pred)

/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The de

6545 36
[0.02777778 0.08333333 0.02777778 ... 0.22222222 0.11111111 0.        ]
[[1 1 0 1 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]]
Average overall accuracy perceptron 94.15% 

Average overall accuracy random forest 94.25% 



### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

### 6. Improve your model
Use grid search to find better parameters. 

In [39]:
print(pipeline.get_params())
#parameters = 

#cv = 
model = build_model()
model.fit(X_train, y_train)
y_pred_perceptron2 = model.predict(X_test)


#f1 = multioutput_fscore(y_test,y_pred, beta = 1)
overall_accuracy_perceptron2 = (y_pred_perceptron2 == y_test).mean().mean()
print('Average overall accuracy perceptron {0:.2f}% \n'.format(overall_accuracy_perceptron*100))
filename = 'classifier.sav'
pickle.dump(model, open(filename, 'wb'))

{'memory': None, 'steps': [('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=<function tokenize at 0x1a360f91e0>, vocabulary=None)), ('tfidf', TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)), ('clf', MultiOutputClassifier(estimator=MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(6, 3), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=1, shuffle=True, solver='lbfgs', tol=0.0001,
       validation_fraction=0.1, verbos

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed: 62.3min
[Parallel(n_jobs=-1)]: Done  48 out of  48 | elapsed: 126.9min finished


Average overall accuracy perceptron 93.61% 



### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [47]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):
# from the lesson's notebook
    def starting_verb(self, text):
        try:
            sentence_list = sent_tokenize(text)
            for sentence in sentence_list:
                pos_tags = pos_tag(tokenize(sentence))
                first_word, first_tag = pos_tags[0]
                if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                    return True
        except:
            return False
                    
        return False

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [49]:
def build_model_verb():
    pipeline = Pipeline([
        ('features', FeatureUnion([
            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('starting_verb', StartingVerbExtractor())
        ])),

        ('clf', MultiOutputClassifier(MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(6, 3), random_state=1)))
        ])
    parameters ={
        'clf__estimator__hidden_layer_sizes':((32,3) ,(32,4),(64,3)),
#        'vect__ngram_range': ((1, 1), (1, 2)),
#        'vect__max_df': (0.5, 0.75, 1.0),
#        'vect__max_features': (5000, 10000),
#        'clf__estimator__alpha' : (0.00001 ,0.0001 , 0.001 )

    }
    cv  = GridSearchCV(pipeline, param_grid=parameters,verbose=3,n_jobs=3,cv=3)
    return cv
model_verb = build_model_verb()
model_verb.fit(X_train, y_train)
y_pred_perceptron_verb = model.predict(X_test)
overall_accuracy_perceptron_verb = (y_pred_perceptron_verb == y_test).mean().mean()
print('Average overall accuracy perceptron {0:.2f}% \n'.format(overall_accuracy_perceptron_verb*100))


Fitting 3 folds for each of 3 candidates, totalling 9 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   9 out of   9 | elapsed: 75.5min finished


Average overall accuracy perceptron 93.81% 



### 9. Export your model as a pickle file

In [ ]:
filename = 'classifier.sav'
pickle.dump(model, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.